In [3]:
import numpy as np
import json
from benlp.llms import Chat
from scipy.optimize import linprog

## The Optimization Function

In [4]:
def linear_optimization(c, A_ub, b_ub, A_eq=None, b_eq=None, bounds=None):
    """
    Solves a linear programming optimization problem using the highs algorithm.

    Parameters:
    c (list): The coefficients of the linear objective function to be minimized.
    A_ub (list): The inequality constraint matrix. Each row represents a constraint.
    b_ub (list): The inequality constraint vector. Each element represents the upper bound of the corresponding constraint.
    A_eq (list, optional): The equality constraint matrix. Each row represents a constraint.
    b_eq (list, optional): The equality constraint vector. Each element represents the required value of the corresponding constraint.
    bounds (list, optional): A list of bounds for each variable in the form (min, max).

    Returns:
    dict: A dictionary containing the optimal solution, the minimized objective function value, and the status of the optimization.
    """
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    
    return {
        'x': result.x,
        'fun': result.fun,
        'slack': result.slack,
        'success': result.success,
        'message': result.message,
        'nit': result.nit
    }

## The Args Parser

In [5]:
def args_parser(json_string):
    data = json.loads(json_string)
    c = np.array(data.get("c"))
    A_ub = np.array(data.get("A_ub"))
    b_ub = np.array(data.get("b_ub"))
    A_eq = np.array(data.get("A_eq")) if data.get("A_eq") is not None else None
    b_eq = np.array(data.get("b_eq")) if data.get("b_eq") is not None else None
    bounds = np.array(data.get("bounds")) if data.get("bounds") is not None else None
    args = c, A_ub, b_ub, A_eq, b_eq, bounds
    return args

## The Prompt Components

In [6]:
code_string = """
from scipy.optimize import linprog

def linear_optimization(c, A_ub, b_ub, A_eq=None, b_eq=None, bounds=None):

    Solves a linear programming optimization problem using the highs algorithm.

    Parameters:
    c (list): The coefficients of the linear objective function to be minimized.
    A_ub (list): The inequality constraint matrix. Each row represents a constraint.
    b_ub (list): The inequality constraint vector. Each element represents the upper bound of the corresponding constraint.
    A_eq (list, optional): The equality constraint matrix. Each row represents a constraint.
    b_eq (list, optional): The equality constraint vector. Each element represents the required value of the corresponding constraint.
    bounds (list, optional): A list of bounds for each variable in the form (min, max).

    Returns:
    dict: A dictionary containing the optimal solution, the minimized objective function value, and the status of the optimization.

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    
    return {
        'x': result.x,
        'fun': result.fun,
        'slack': result.slack,
        'success': result.success,
        'message': result.message,
        'nit': result.nit
    }
"""

In [7]:
docs = """
scipy.optimize.linprog
scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None)[source]
Linear programming: minimize a linear objective function subject to linear equality and inequality constraints.

Linear programming solves problems of the following form:

where 
 is a vector of decision variables; 
, 
, 
, 
, and 
 are vectors; and 
 and 
 are matrices.

Alternatively, that’s:

minimize:

c @ x
such that:

A_ub @ x <= b_ub
A_eq @ x == b_eq
lb <= x <= ub
Note that by default lb = 0 and ub = None unless specified with bounds.

Parameters:
c1-D array
The coefficients of the linear objective function to be minimized.

A_ub2-D array, optional
The inequality constraint matrix. Each row of A_ub specifies the coefficients of a linear inequality constraint on x.

b_ub1-D array, optional
The inequality constraint vector. Each element represents an upper bound on the corresponding value of A_ub @ x.

A_eq2-D array, optional
The equality constraint matrix. Each row of A_eq specifies the coefficients of a linear equality constraint on x.

b_eq1-D array, optional
The equality constraint vector. Each element of A_eq @ x must equal the corresponding element of b_eq.

boundssequence, optional
A sequence of (min, max) pairs for each element in x, defining the minimum and maximum values of that decision variable. Use None to indicate that there is no bound. By default, bounds are (0, None) (all decision variables are non-negative). If a single tuple (min, max) is provided, then min and max will serve as bounds for all decision variables.

methodstr, optional
The algorithm used to solve the standard form problem. ‘highs’ (default), ‘highs-ds’, ‘highs-ipm’, ‘interior-point’ (legacy), ‘revised simplex’ (legacy), and ‘simplex’ (legacy) are supported. The legacy methods are deprecated and will be removed in SciPy 1.11.0.

callbackcallable, optional
If a callback function is provided, it will be called at least once per iteration of the algorithm. The callback function must accept a single scipy.optimize.OptimizeResult consisting of the following fields:

x1-D array
The current solution vector.

funfloat
The current value of the objective function c @ x.

successbool
True when the algorithm has completed successfully.

slack1-D array
The (nominally positive) values of the slack, b_ub - A_ub @ x.

con1-D array
The (nominally zero) residuals of the equality constraints, b_eq - A_eq @ x.

phaseint
The phase of the algorithm being executed.

statusint
An integer representing the status of the algorithm.

0 : Optimization proceeding nominally.

1 : Iteration limit reached.

2 : Problem appears to be infeasible.

3 : Problem appears to be unbounded.

4 : Numerical difficulties encountered.

nitint
The current iteration number.

messagestr
A string descriptor of the algorithm status.

Callback functions are not currently supported by the HiGHS methods.

optionsdict, optional
A dictionary of solver options. All methods accept the following options:

maxiterint
Maximum number of iterations to perform. Default: see method-specific documentation.

dispbool
Set to True to print convergence messages. Default: False.

presolvebool
Set to False to disable automatic presolve. Default: True.

All methods except the HiGHS solvers also accept:

tolfloat
A tolerance which determines when a residual is “close enough” to zero to be considered exactly zero.

autoscalebool
Set to True to automatically perform equilibration. Consider using this option if the numerical values in the constraints are separated by several orders of magnitude. Default: False.

rrbool
Set to False to disable automatic redundancy removal. Default: True.

rr_methodstring
Method used to identify and remove redundant rows from the equality constraint matrix after presolve. For problems with dense input, the available methods for redundancy removal are:

“SVD”:
Repeatedly performs singular value decomposition on the matrix, detecting redundant rows based on nonzeros in the left singular vectors that correspond with zero singular values. May be fast when the matrix is nearly full rank.

“pivot”:
Uses the algorithm presented in [5] to identify redundant rows.

“ID”:
Uses a randomized interpolative decomposition. Identifies columns of the matrix transpose not used in a full-rank interpolative decomposition of the matrix.

None:
Uses “svd” if the matrix is nearly full rank, that is, the difference between the matrix rank and the number of rows is less than five. If not, uses “pivot”. The behavior of this default is subject to change without prior notice.

Default: None. For problems with sparse input, this option is ignored, and the pivot-based algorithm presented in [5] is used.

For method-specific options, see show_options('linprog').

x01-D array, optional
Guess values of the decision variables, which will be refined by the optimization algorithm. This argument is currently used only by the ‘revised simplex’ method, and can only be used if x0 represents a basic feasible solution.

integrality1-D array or int, optional
Indicates the type of integrality constraint on each decision variable.

0 : Continuous variable; no integrality constraint.

1 : Integer variable; decision variable must be an integer within bounds.

2 : Semi-continuous variable; decision variable must be within bounds or take value 0.

3 : Semi-integer variable; decision variable must be an integer within bounds or take value 0.

By default, all variables are continuous.

For mixed integrality constraints, supply an array of shape c.shape. To infer a constraint on each decision variable from shorter inputs, the argument will be broadcasted to c.shape using np.broadcast_to.

This argument is currently used only by the 'highs' method and ignored otherwise.

Returns:
resOptimizeResult
A scipy.optimize.OptimizeResult consisting of the fields below. Note that the return types of the fields may depend on whether the optimization was successful, therefore it is recommended to check OptimizeResult.status before relying on the other fields:

x1-D array
The values of the decision variables that minimizes the objective function while satisfying the constraints.

funfloat
The optimal value of the objective function c @ x.

slack1-D array
The (nominally positive) values of the slack variables, b_ub - A_ub @ x.

con1-D array
The (nominally zero) residuals of the equality constraints, b_eq - A_eq @ x.

successbool
True when the algorithm succeeds in finding an optimal solution.

statusint
An integer representing the exit status of the algorithm.

0 : Optimization terminated successfully.

1 : Iteration limit reached.

2 : Problem appears to be infeasible.

3 : Problem appears to be unbounded.

4 : Numerical difficulties encountered.

nitint
The total number of iterations performed in all phases.

messagestr
A string descriptor of the exit status of the algorithm.

See also

show_options
Additional options accepted by the solvers.
"""

In [43]:
json_rules = """
JSON Schema is a declarative way of writing validation rules. It contains all the steps that are necessary to be performed in order to validate something. The format used to write these steps is, of course, JSON itself.

Opis JSON Schema is the software/library (for PHP) that performs those steps and tells you the validation status.

Data types
Because JSON Schema is written in JSON format, it supports all JSON types plus an addition: the integer type, which is a subtype of the number type.

string - represents a string/text, e.g. "a string", "other string"
number - represents an integer or a float, e.g. -5, 10, -5.8, 10.2
integer - represents an integer, e.g. -100, 125, 0
boolean - represents a boolean value, e.g. true or false
null - indicates that a value is missing, e.g. null
object - a key-value map, where the key must be a string and the value can be any type, e.g. {"key": "value", "other-key": 5}
array - an ordered list of any data types, e.g. [1, -2.5, "some string", null]
Document structure
A valid json schema document must be a JSON object or a boolean value. If it is a boolean, then the validation status is indicated by the value of the boolean: true - valid, false - invalid. If it is an object, then it must contain the steps needed for validation. These steps come in the form of keywords and every keyword has a specific meaning. Keywords are applied to data starting from the root of the document schema and descend to their children.

Here are some examples

true
Always valid.

false
Always invalid.

{}
Always valid because there are no steps defined.

{
  "type": "string"
}
Validation status is keyword dependent (in this case) the data is valid only if it holds a string.

Some keywords are purely decorative, like metadata keywords, which just describe the author intent. Others are for identifying a document or a subschema, and the rest of them are for validity checks. Usually keywords work independently and there are only a few exceptions.

$schema keyword
This keyword is used to specify the desired schema version. The value of this keyword must be a string representing an URI.

Currently the supported URIs are:

https://json-schema.org/draft/2020-12/schema - latest version
https://json-schema.org/draft/2019-09/schema - previous version
http://json-schema.org/draft-07/schema#
http://json-schema.org/draft-06/schema#
This keyword is not required and if it is missing, the URI of the latest schema version will be used instead.

Not all keywords all available on all drafts, so we recommend using this keyword to set the right context.

$id keyword
This keyword is used to specify an unique ID for a document or a document subschemas. The value of this keyword must be a string representing an URI. All subschema IDs are resolved relative to the document’s ID. It is not a required keyword, but we recommend you using it, as a best practice.

The usage of this keyword will be covered in the next chapters.

$anchor keyword
This keyword is used to define fragment identifiers.

The two schemas below are equivalent.

{
 "$id": "http://example.com",
 
 "$ref": "#mail",
 
 "$defs": {
  "mail": {
   "$anchor": "mail",
   "format": "email"
  }
 }
}
definitions keyword
This keyword does not directly validate data, but it contains a map of validation schemas. The value of this keyword can be anything. This keyword is not required.

$defs keyword
Same as definitions keyword (standardized staring with draft-2019-09).

Metadata keywords
These keywords are not used for validation, but to describe the validation schema and how it works. All keywords are optional.

title
Contains a short description about the validation. The value of this keyword must be a string.

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "http://example.com/number.json#",
  "title": "Test if it is a number",
  
  "type": "number"
}
description
Contains a long description about the validation. The value of this keyword must be a string.

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "http://example.com/number.json#",
  "title": "Test if it is a number",
  "description": "A data is considered number if it is an integer or a float.",
  
  "type": "number"
}
examples
Contains a list of valid examples. The value of this keyword must be an array.

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "http://example.com/number.json#",
  "title": "Test if it is a number",
  "description": "A data is considered a number if is an integer or a float.",
  "examples": [-5.10, -2, 0, 5, 8.10],
  
  "type": "number"
}
$comment
Contains an observation about the schema. The value of this keyword must be a string.

{
  "$comment": "We should replace this broken regex with format: email.",
  
  "type": "string",
  "pattern": "[a-zA-Z0-9\\.]+@[a-zA-Z0-9]+\\.[a-zA-Z]{2,3}"
}
JSON Schema examples
For most of the basic examples we will not use $schema, $id and metadata keywords.

Don’t worry if you don’t exactly understand every keyword, they are presented in depth in the next chapters.

Validating a simple user
{
  "type": "object",
  "properties": {
    "name": {
      "type": "string"
    },
    "email": {
      "type": "string",
      "format": "email"
    },
    "age": {
      "type": "integer",
      "minimum": 18,
      "maximum": 150
    }
  },
  "required": ["email"]
}
Validating a list
{
  "type": "array",
  "minItems": 2,
  "items": {
    "type": ["string", "number"]
  }
}

You can only use "null", not "None"
"""

Example problems

In [46]:
word_problems = [
    "The minimum daily req for vitamin A and b are 100 and 200 respectively. pizza has 10 vitamin A and 30 vitamin B. root beer has 20 vitamin A and 10 vitamin b. pizza costs $10 and root beet costs $5. Maximize profit",
    "A farming cooperative mixes two brands of cattle feed. Brand X costs $25 per bag and contains 2 units of nutritional element A, 2 units of element B, and 2 units of element C. Brand Y costs $20 per bag and contains 1 unit of nutritional element A, 9 units of element B, and 3 units of element C. Find the number of bags of each brand that should be mixed to produce a mixture having a minimum cost per bag. The minimum requirements of nutrients A, B, and C are 12 units, 36 units, and 24 units, respectively.",
    "A local family-owned plastic cup manufacturer wants to optimize their production mix in order to maximize their profit. They produce personalized beer mugs and champagne glasses. The profit on a case of beer mugs is $25 while the profit on a case of champagne glasses is $20. The cups are manufactured with a machine called a plastic extruder which feeds on plastic resins. Each case of beer mugs requires 20 lbs. of plastic resins to produce while champagne glasses require 12 lbs. per case. The daily supply of plastic resins is limited to at most 1800 pounds. About 15 cases of either product can be produced per hour. At the moment the family wants to limit their work day to 8 hours.",
    "A company makes two products (X and Y) using two machines (A and B). Each unit of X that is produced requires 50 minutes processing time on machine A and 30 minutes processing time on machine B. Each unit of Y that is produced requires 24 minutes processing time on machine A and 33 minutes processing time on machine B. At the start of the current week there are 30 units of X and 90 units of Y in stock. Available processing time on machine A is forecast to be 40 hours and on machine B is forecast to be 35 hours. The demand for X in the current week is forecast to be 75 units and for Y is forecast to be 95 units. Company policy is to maximise the combined sum of the units of X and the units of Y in stock at the end of the week. Formulate the problem of deciding how much of each product to make in the current week as a linear program."
]

more problems [here](http://people.brunel.ac.uk/~mastjjb/jeb/or/morelp.html)

In [16]:
problem_1 = "The minimum daily req for vitamin A and b are 100 and 200 respectively. pizza has 10 vitamin A and 30 vitamin B. root beer has 20 vitamin A and 10 vitamin b. pizza costs $10 and root beet costs $5. Maximize profit"
args_1 = """{
            "c": [-10,-5],
            "A_ub": [[10, 20], [30, 10]],
            "b_ub": [-100, -200],
            "A_eq": null,
            "b_eq": null,
            "bounds": [[0, null], [0, null]]
        }"""
notes_1 = "The `c` paramater was negated because the problem statement was to maximize profit. The default is to minimize the objective function. Nothing else is negated."

Prompts

In [48]:
prompt1 = f"""Given the word problem: {word_problems[0]}, identify the following:
1. The type of problem (maximization or minimization)
2. The objective function.
3. The constraints
4. The variables
5. The bounds

Output the data above.
"""
prompt2 = f"""Here are the scipy.linprog docs:\n{docs}\n""" + f"""Here is the code to run the function:
{code_string}\n""" + f"""Here are the valid json rules: {json_rules}\n""" + """
INSTRUCTION:
Output a json object with the required arguments to run the function.
Output valid json only. """ + f"""

EXAMPLE Args: {args_1}

Remember, make sure the signs are correct. The default is to minimize the objective function. Nothing else is negated. So, c is negated if the problem is to maximize profit. Nothing else is ever negated. Make sure you didnt accidentally negate the constraints from the context.

YOUR OUTPUT:
"""

In [49]:
print("Analyzing problem and extracting arguments...")
chat = Chat(model="gpt-4", temperature=0)
intermediate = chat(prompt1)['response'].strip()
print("Intermediate Response:\n", intermediate)

unparsed_args = chat(prompt2)['response'].strip()
print("Unparsed args:\n", unparsed_args)
print("Parsing args...")
parsed_args = args_parser(unparsed_args)

print("Running linear optimization...")
result = linear_optimization(*parsed_args)
print("Result:", result)

print("Interpreting result...")
res = chat(f"Result: {result} \n\n INSTRUCTION: Interperet the result in the context of the problem. Only use data from the above result, do not infer any other data.")['response'].strip()
print("Final Response:\n\n", res)

Analyzing problem and extracting arguments...
Intermediate Response:
 1. The type of problem: Maximization (Maximize profit)

2. The objective function: P = 10x + 5y (where P is the profit, x is the number of pizzas, and y is the number of root beers)

3. The constraints: 
   10x + 20y ≥ 100 (Vitamin A requirement)
   30x + 10y ≥ 200 (Vitamin B requirement)
   x ≥ 0, y ≥ 0 (Non-negativity constraint)

4. The variables: 
   x = number of pizzas
   y = number of root beers

5. The bounds: 
   x ≥ 0
   y ≥ 0
Unparsed args:
 {
    "c": [-10, -5],
    "A_ub": [[10, 20], [30, 10]],
    "b_ub": [100, 200],
    "A_eq": null,
    "b_eq": null,
    "bounds": [[0, null], [0, null]]
}
Parsing args...
Running linear optimization...
Result: {'x': array([6., 2.]), 'fun': -70.0, 'slack': array([0., 0.]), 'success': True, 'message': 'Optimization terminated successfully. (HiGHS Status 7: Optimal)', 'nit': 2}
Interpreting result...
Final Response:

 The optimal solution to maximize profit is to sell 6 p